# 使用分区键


本指南将指导您使用分区键加速从集合中检索数据。

概述
Milvus中的分区键允许将传入的实体基于其各自的分区键值分布到不同的分区中。这允许具有相同键值的实体被分组在一个分区中，这反过来又通过避免在按键字段过滤时扫描不相关分区的需要来加速搜索性能。与传统的过滤方法相比，分区键可以大大提高查询性能。

您可以使用分区键来实现多租户。有关多租户的详细信息，请阅读多租户。

此页面上的代码片段使用新的MilvusClient（Python）与Milvus交互。其他语言的新MilvusClient SDK将在未来的更新中发布。

启用分区键
为了演示分区键的使用，我们将继续使用包含5,000多篇文章的示例数据集，发布字段将用作分区键。

在下面的示例代码中，num_partitions确定将创建的分区数量。默认情况下，它设置为64。我们建议您保留默认值。


In [1]:
import json, time
import os

from pymilvus import MilvusClient, DataType

In [2]:
COLLECTION_NAME="medium_articles_2020" # Set your collection name

# 1. Connect to cluster
client = MilvusClient(
    uri="http://localhost:19530"
)

# 2. Define collection schema
schema = client.create_schema(
    auto_id=True,
    partition_key_field="publication",
    num_partitions=64 # Number of partitions. Defaults to 64.
)

schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="title", datatype=DataType.VARCHAR, max_length=512)
schema.add_field(field_name="title_vector", datatype=DataType.FLOAT_VECTOR, dim=768)
schema.add_field(field_name="link", datatype=DataType.VARCHAR, max_length=512)
schema.add_field(field_name="reading_time", datatype=DataType.INT64)
schema.add_field(field_name="publication", datatype=DataType.VARCHAR, max_length=512)
schema.add_field(field_name="claps", datatype=DataType.INT64)
schema.add_field(field_name="responses", datatype=DataType.INT64)

{'auto_id': True, 'description': '', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'title', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 512}}, {'name': 'title_vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 768}}, {'name': 'link', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 512}}, {'name': 'reading_time', 'description': '', 'type': <DataType.INT64: 5>}, {'name': 'publication', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 512}}, {'name': 'claps', 'description': '', 'type': <DataType.INT64: 5>}, {'name': 'responses', 'description': '', 'type': <DataType.INT64: 5>}], 'enable_dynamic_field': False}

定义字段后，设置其他必要的参数。

In [3]:
# 3. Define index parameters
index_params = client.prepare_index_params()

index_params.add_index(
    field_name="title_vector",
    index_type="AUTOINDEX",
    metric_type="L2"
)

最后，您可以创建一个集合。

In [5]:
# 4. Create a collection
client.create_collection(
    collection_name=COLLECTION_NAME,
    schema=schema,
    index_params=index_params
)

插入数据
集合准备就绪后，开始插入数据，如下所示：

准备数据

In [ ]:
with open(DATASET_PATH) as f:
    data = json.load(f)
    list_of_rows = data['rows']

    data_rows = []
    for row in list_of_rows:
        # Remove the id field because the primary key has auto_id enabled.
        del row['id']
        # Other keys except the title and title_vector fields in the row 
        # will be treated as dynamic fields.
        data_rows.append(row)

插入数据

In [ ]:
# 7. Insert data
res = client.insert(
    collection_name=COLLECTION_NAME,
    data=data_rows,
)

time.sleep(5000)


使用分区键
索引并加载集合以及插入的数据后，可以使用分区键进行相似性搜索。

注意到

要使用分区键执行相似性搜索，您应该在搜索请求的布尔表达式中包含以下内容之一：

In [ ]:
expr='<partition_key>=="xxxx"'

expr='<partition_key> in ["xxx", "xxx"]'

将<partition_key>替换为指定为分区键的字段的名称。

In [6]:
res = client.search(
    collection_name=COLLECTION_NAME,
    data=[data_rows[0]['title_vector']],
    filter='claps > 30 and reading_time < 10',
    limit=3,
    output_fields=["title", "reading_time", "claps"],
    search_params={"metric_type": "L2", "params": {}}
)

print(res)

NameError: name 'data_rows' is not defined


用例
要实现更好的搜索性能并启用多租户，您可以利用分区键功能。这可以通过为每个实体分配特定于租户的值作为分区键字段来完成。在搜索或查询集合时，可以通过在布尔表达式中包含分区键字段，按特定于租户的值过滤实体。这种方法确保了租户的数据隔离，并避免扫描不必要的分区。